In [1]:
import random
import pandas as pd
import numpy as np
import collections

In [2]:
def initialize_data():
    customer_demands_df = pd.read_csv("demands.csv")
    store_stocks_df = pd.read_csv("stocks.csv")

    vehicle_capacity = 20

    return customer_demands_df, store_stocks_df, vehicle_capacity

def merge_products(dict1, dict2):
    """Merges two dictionaries of products and their quantities."""
    merged = collections.defaultdict(int)
    for p, q in dict1.items():
        merged[p] += q
    for p, q in dict2.items():
        merged[p] += q
    return dict(merged)

def simulate_route_capacity(route, capacity):
    """
    Simulates a route to check if vehicle capacity is exceeded at any point.
    Returns True if capacity is respected, False otherwise.
    Also returns the maximum load achieved during the simulation.
    """
    current_load_dict = collections.defaultdict(int)
    max_load_achieved = 0
    
    for action in route:
        if action["type"] == "pickup":
            # For pickups, the action["products"] represents the items collected at this stop.
            # These are added to the vehicle's current load.
            for product, qty in action["products"].items():
                current_load_dict[product] += qty
        elif action["type"] == "delivery":
            # For deliveries, products are removed from the vehicle's load.
            for product, qty in action["products"].items(): # qty will be negative
                current_load_dict[product] += qty
                if current_load_dict[product] <= 0:
                    del current_load_dict[product] # Remove if quantity is zero or less
        
        current_total_load = sum(current_load_dict.values())
        if current_total_load > max_load_achieved:
            max_load_achieved = current_total_load
            
        if current_total_load > capacity:
            return False, max_load_achieved # Capacity exceeded
            
    return True, max_load_achieved

def check_and_assign_to_existing_vehicle(vehicle, customer_order, capacity):
    """
    Tries to assign a customer_order to an existing vehicle.
    Modifies the vehicle in place if successful.
    Returns True if assigned, False otherwise.
    """
    customer_id = customer_order["CustomerID"]
    sourcing_details = customer_order["sourcing_details"]
    quantity_needed = customer_order["quantity_needed"]

    # --- Stage 1: Preliminary checks and identify potential operations ---
    # Stores what this customer needs from each store, and if it's a merge with an existing stop
    customer_pickup_operations_info = [] 
    
    for store_id, products_for_cust_from_store in sourcing_details.items():
        is_merge = store_id in vehicle["visited_stores_for_pickup"]
        
        # Products currently picked up by this vehicle from this store (if any)
        current_vehicle_pickup_at_store = vehicle["visited_stores_for_pickup"].get(store_id, {})
        
        # What the total pickup from this store would be if this customer is added
        combined_pickup_at_store = merge_products(current_vehicle_pickup_at_store, products_for_cust_from_store)

        # Constraint: Total picked up from a single store stop cannot exceed capacity
        if sum(combined_pickup_at_store.values()) > capacity:
            # This specific check applies if we are merging, or if it's a new store but this customer's demand from it is too high.
            # The problem states: "M2 has 6 P1 and if we merge it it will be 11 it greater than capacity so we can't merge it"
            # This implies the check is on the *total quantity of the combined pickup at the store*.
            if is_merge or sum(products_for_cust_from_store.values()) > capacity : # check for new store too
                 return False # Cannot assign due to store-specific pickup capacity violation

        customer_pickup_operations_info.append({
            "store_id": store_id,
            "products_for_this_customer": products_for_cust_from_store, 
            "is_merge": is_merge,
            "final_pickup_at_store_if_assigned": combined_pickup_at_store 
        })

    # --- Stage 2: Construct the proposed new route for the vehicle ---
    # The route is built by:
    # 1. Taking the vehicle's current route.
    # 2. Updating product quantities for any pickup actions that are being merged.
    # 3. Appending new pickup actions (for stores not previously visited by this vehicle for other customers).
    # 4. Appending the delivery action for the current customer.

    proposed_route = []
    
    # Part 1: Process existing route, updating merged pickups
    for action in vehicle["route"]:
        action_copy = {k: (v.copy() if isinstance(v, dict) else v) for k, v in action.items()} # Shallow copy for modification
        
        if action_copy["type"] == "pickup":
            store_id_in_action = action_copy["store"]
            # Check if this pickup is for a store that this new customer also needs (and is a merge)
            merge_info = next((op_info for op_info in customer_pickup_operations_info 
                               if op_info["store_id"] == store_id_in_action and op_info["is_merge"]), None)
            if merge_info:
                action_copy["products"] = merge_info["final_pickup_at_store_if_assigned"]
        proposed_route.append(action_copy)

    # Part 2: Add new pickup actions for this customer (stores not previously on vehicle's route)
    for op_info in customer_pickup_operations_info:
        if not op_info["is_merge"]: # This is a new store stop for this vehicle
            proposed_route.append({
                "type": "pickup",
                "store": op_info["store_id"],
                "products": op_info["products_for_this_customer"] # Just what this customer needs
            })
            
    # Part 3: Add delivery action for the current customer
    proposed_route.append({
        "type": "delivery",
        "customer": customer_id,
        "products": quantity_needed.copy()
    })

    # --- Stage 3: Simulate the proposed route for overall vehicle capacity ---
    capacity_ok, max_load = simulate_route_capacity(proposed_route, capacity)
    if not capacity_ok:
        return False # Overall vehicle capacity exceeded along the proposed route

    # --- Stage 4: If all checks pass, commit changes to the vehicle ---
    vehicle["route"] = proposed_route
    vehicle["assigned_customers"].append(customer_id)
    
    # Update the vehicle's record of total products picked up from each store
    new_visited_stores_summary = collections.defaultdict(lambda: collections.defaultdict(int))
    for action in vehicle["route"]:
        if action["type"] == "pickup":
            store_id = action["store"]
            # action["products"] now represents the total picked up at this store stop in the new route
            new_visited_stores_summary[store_id] = action["products"].copy() 
            
    vehicle["visited_stores_for_pickup"] = {k: dict(v) for k, v in new_visited_stores_summary.items()}
    
    return True

def create_new_vehicle_for_customer(customer_order, capacity, vehicle_id):
    """
    Creates a new vehicle for a single customer_order.
    Returns the new vehicle object if successful, None otherwise.
    """
    customer_id = customer_order["CustomerID"]
    sourcing_details = customer_order["sourcing_details"]
    quantity_needed = customer_order["quantity_needed"]

    new_vehicle_route = []
    current_load_for_new_vehicle = collections.defaultdict(int)
    max_load_for_new_vehicle = 0

    # Add pickup actions
    for store_id, products_to_pickup in sourcing_details.items():
        # Check if this single pickup exceeds capacity (for a new vehicle, each store is a new stop)
        if sum(products_to_pickup.values()) > capacity:
            print(f"Error: Customer {customer_id}'s demand from store {store_id} ({sum(products_to_pickup.values())}) exceeds vehicle capacity ({capacity}). Cannot create vehicle.")
            return None 
        
        new_vehicle_route.append({
            "type": "pickup",
            "store": store_id,
            "products": products_to_pickup.copy()
        })
        for p, q in products_to_pickup.items():
            current_load_for_new_vehicle[p] += q
        
        current_total_load = sum(current_load_for_new_vehicle.values())
        if current_total_load > max_load_for_new_vehicle:
             max_load_for_new_vehicle = current_total_load
        if current_total_load > capacity: # Should be caught by sum(products_to_pickup.values()) check earlier for single pickups
            print(f"Error: Customer {customer_id}'s cumulative demand ({current_total_load}) exceeds vehicle capacity ({capacity}) during pickup phase. Cannot create vehicle.")
            return None


    # Add delivery action
    new_vehicle_route.append({
        "type": "delivery",
        "customer": customer_id,
        "products": quantity_needed.copy()
    })
    # Simulate after delivery to ensure load calculation is consistent, though max load usually occurs before/at last pickup
    for p, q in quantity_needed.items(): # q is negative
        current_load_for_new_vehicle[p] += q
        if current_load_for_new_vehicle[p] <= 0:
            del current_load_for_new_vehicle[p]

    # Final check on overall capacity using the simulation function (more robust)
    capacity_ok, max_load_sim = simulate_route_capacity(new_vehicle_route, capacity)
    if not capacity_ok:
        print(f"Error: Customer {customer_id}'s order route simulation failed capacity check ({max_load_sim} > {capacity}). Cannot create vehicle.")
        return None

    visited_stores_summary = collections.defaultdict(lambda: collections.defaultdict(int))
    for action in new_vehicle_route:
        if action["type"] == "pickup":
            visited_stores_summary[action["store"]] = action["products"].copy()

    return {
        "id": vehicle_id,
        "route": new_vehicle_route,
        "assigned_customers": [customer_id],
        "visited_stores_for_pickup": {k: dict(v) for k, v in visited_stores_summary.items()} # Stores total picked up from each store by this vehicle
    }

def solve_vrpdp_custom(all_customer_orders, vehicle_capacity):
    """
    Main solver function.
    Processes customer orders and assigns them to vehicles.
    """
    vehicles = []
    vehicle_id_counter = 0
    
    # Use deepcopy for all_customer_orders if it's going to be modified or contains mutable sub-elements
    # that should not be shared across processing steps if the original list is reused.
    # For this logic, we are reading from it, so a shallow copy of the list is fine,
    # but internal dicts should be copied when used.

    for customer_order in all_customer_orders:
        customer_id = customer_order["CustomerID"]
        assigned_to_existing = False
        for vehicle in vehicles:
            # Create a deepcopy of vehicle to attempt assignment. If it fails, original vehicle is unchanged.
            # This is complex. Let's modify check_and_assign to handle its own rollback or commit.
            # For now, the function modifies in place if successful.
            if check_and_assign_to_existing_vehicle(vehicle, customer_order, vehicle_capacity):
                print(f"Customer {customer_id} assigned to existing vehicle {vehicle['id']}.")
                assigned_to_existing = True
                break
        
        if not assigned_to_existing:
            vehicle_id_counter += 1
            new_vehicle_id_str = f"V{vehicle_id_counter}"
            print(f"Attempting to create new vehicle {new_vehicle_id_str} for customer {customer_id}...")
            new_vehicle = create_new_vehicle_for_customer(customer_order, vehicle_capacity, new_vehicle_id_str)
            if new_vehicle:
                vehicles.append(new_vehicle)
                print(f"Customer {customer_id} assigned to new vehicle {new_vehicle['id']}.")
            else:
                print(f"Could not assign customer {customer_id} to any vehicle or create a new one.")
                # Optionally, handle unserviceable customers (e.g., add to a separate list)

    return vehicles


In [3]:
demands_df, stocks_df, capacity = initialize_data()

In [4]:
customer_ids = list(demands_df["CustomerID"].unique())
random.shuffle(customer_ids)
customer_ids

['C5', 'C3', 'C6', 'C8', 'C4', 'C7', 'C2', 'C10', 'C9', 'C1']

In [5]:
all_T = []
for customer_id in customer_ids:
    customer_demands = demands_df[demands_df["CustomerID"] == customer_id]

    customer_T = {
        "CustomerID": customer_id,
        "sourcing_details": {},
        "quantity_needed": {},
    }

    for _, demand in customer_demands.iterrows():
        product_id = demand["ProductID"]
        quantity_needed = demand["Quantity"]
        sourced_for_this_product = 0

        if product_id not in customer_T["quantity_needed"]:
            customer_T["quantity_needed"][product_id] = 0
        customer_T["quantity_needed"][product_id] = (
            customer_T["quantity_needed"].get(product_id, 0) - quantity_needed
        )

        stores_df = stocks_df[
            (stocks_df["ProductID"] == product_id) & (stocks_df["StockQuantity"] > 0)
        ].copy()

        store_indices = list(stores_df.index)
        random.shuffle(store_indices)

        for store_idx in store_indices:
            if sourced_for_this_product >= quantity_needed:
                break

            store_id = stores_df.loc[store_idx, "StoreID"]
            original_stock_idx_series = stocks_df[
                (stocks_df["StoreID"] == store_id)
                & (stocks_df["ProductID"] == product_id)
            ].index

            if original_stock_idx_series.empty:
                continue

            original_stock_idx = original_stock_idx_series[0]
            stock_in_this_store = stores_df.loc[original_stock_idx, "StockQuantity"]

            quantity_to_take = min(
                quantity_needed - sourced_for_this_product, stock_in_this_store
            )

            if quantity_to_take > 0:
                if store_id not in customer_T["sourcing_details"]:
                    customer_T["sourcing_details"][store_id] = {}
                customer_T["sourcing_details"][store_id][product_id] = (
                    customer_T["sourcing_details"][store_id].get(product_id, 0)
                    + quantity_to_take
                )
                sourced_for_this_product += quantity_to_take
                stocks_df.loc[original_stock_idx, "StockQuantity"] -= quantity_to_take

    all_T.append(customer_T)

random.shuffle(all_T)
all_T

[{'CustomerID': 'C6',
  'sourcing_details': {'M2': {'P7': 9, 'P12': 1}},
  'quantity_needed': {'P7': -9, 'P12': -1}},
 {'CustomerID': 'C4',
  'sourcing_details': {'M3': {'P12': 10}},
  'quantity_needed': {'P12': -10}},
 {'CustomerID': 'C7',
  'sourcing_details': {'M5': {'P6': np.int64(9)}, 'M2': {'P6': np.int64(1)}},
  'quantity_needed': {'P6': -10}},
 {'CustomerID': 'C3',
  'sourcing_details': {'M2': {'P18': 3}, 'M5': {'P10': 5}, 'M1': {'P1': 1}},
  'quantity_needed': {'P18': -3, 'P10': -5, 'P1': -1}},
 {'CustomerID': 'C8',
  'sourcing_details': {'M1': {'P7': 8}},
  'quantity_needed': {'P7': -8}},
 {'CustomerID': 'C2',
  'sourcing_details': {'M1': {'P19': 9}, 'M3': {'P12': 1}},
  'quantity_needed': {'P19': -9, 'P12': -1}},
 {'CustomerID': 'C1',
  'sourcing_details': {'M5': {'P2': np.int64(5), 'P17': 2},
   'M4': {'P2': np.int64(3)}},
  'quantity_needed': {'P2': -8, 'P17': -2}},
 {'CustomerID': 'C10',
  'sourcing_details': {'M2': {'P6': 6}, 'M1': {'P2': 3, 'P14': 1}},
  'quantity_neede

In [6]:
solution_vehicles = solve_vrpdp_custom(all_T, capacity)

Attempting to create new vehicle V1 for customer C6...
Customer C6 assigned to new vehicle V1.
Customer C4 assigned to existing vehicle V1.
Customer C7 assigned to existing vehicle V1.
Customer C3 assigned to existing vehicle V1.
Customer C8 assigned to existing vehicle V1.
Attempting to create new vehicle V2 for customer C2...
Customer C2 assigned to new vehicle V2.
Customer C1 assigned to existing vehicle V2.
Customer C10 assigned to existing vehicle V2.
Attempting to create new vehicle V3 for customer C9...
Customer C9 assigned to new vehicle V3.
Customer C5 assigned to existing vehicle V3.


In [7]:
print("\n--- Final Vehicle Routes ---")
if not solution_vehicles:
    print("No routes generated.")
for v_idx, vehicle_data in enumerate(solution_vehicles):
    print(f"\nVehicle ID: {vehicle_data['id']}")
    print(f"Assigned Customers: {', '.join(vehicle_data['assigned_customers'])}")
    print("Route:")
    for step_idx, step in enumerate(vehicle_data['route']):
        if step['type'] == 'pickup':
            products_str = ', '.join([f"{p_qty} {p_id}" for p_id, p_qty in step['products'].items()])
            print(f"  {step_idx+1}. Pickup from {step['store']}: {products_str}")
        elif step['type'] == 'delivery':
            # For display, make delivered quantities positive
            products_str = ', '.join([f"{-p_qty} {p_id}" for p_id, p_qty in step['products'].items()])
            print(f"  {step_idx+1}. Deliver to {step['customer']}: {products_str}")
    print("Total products picked up by this vehicle from stores:")
    for store, prods in vehicle_data["visited_stores_for_pickup"].items():
        prods_str = ', '.join([f"{qty} {p}" for p,qty in prods.items()])
        print(f"  Store {store}: {prods_str}")


--- Final Vehicle Routes ---

Vehicle ID: V1
Assigned Customers: C6, C4, C7, C3, C8
Route:
  1. Pickup from M2: 9 P7, 1 P12, 1 P6, 3 P18
  2. Deliver to C6: 9 P7, 1 P12
  3. Pickup from M3: 10 P12
  4. Deliver to C4: 10 P12
  5. Pickup from M5: 9 P6, 5 P10
  6. Deliver to C7: 10 P6
  7. Pickup from M1: 1 P1, 8 P7
  8. Deliver to C3: 3 P18, 5 P10, 1 P1
  9. Deliver to C8: 8 P7
Total products picked up by this vehicle from stores:
  Store M2: 9 P7, 1 P12, 1 P6, 3 P18
  Store M3: 10 P12
  Store M5: 9 P6, 5 P10
  Store M1: 1 P1, 8 P7

Vehicle ID: V2
Assigned Customers: C2, C1, C10
Route:
  1. Pickup from M1: 9 P19, 3 P2, 1 P14
  2. Pickup from M3: 1 P12
  3. Deliver to C2: 9 P19, 1 P12
  4. Pickup from M5: 5 P2, 2 P17
  5. Pickup from M4: 3 P2
  6. Deliver to C1: 8 P2, 2 P17
  7. Pickup from M2: 6 P6
  8. Deliver to C10: 6 P6, 3 P2, 1 P14
Total products picked up by this vehicle from stores:
  Store M1: 9 P19, 3 P2, 1 P14
  Store M3: 1 P12
  Store M5: 5 P2, 2 P17
  Store M4: 3 P2
  Store 